In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/pedestrian'

/content/drive/MyDrive/pedestrian


In [ ]:
!pip install youtube_dl

import youtube_dl

link = 'https://www.youtube.com/watch?v=LoAYJNINWmU'

ydl_opts = {}

with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    info_dict = ydl.extract_info(link, download=False)
    video_title = info_dict.get('title', None)

path = f'./{video_title}.mp4'

ydl_opts.update({'outtmpl':path})

with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download([link])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 8.4 MB/s 
[youtube] LoAYJNINWmU: Downloading webpage
[youtube] LoAYJNINWmU: Downloading MPD manifest
[youtube] LoAYJNINWmU: Downloading webpage
[youtube] LoAYJNINWmU: Downloading MPD manifest
[download] Destination: ./Heavy Traffic on the roads of Delhi.f135.mp4
[download] 100% of 6.39MiB in 01:27
[download] Destination: ./Heavy Traffic on the roads of Delhi.mp4.f140
[download] 100% of 1019.14KiB in 00:13
[ffmpeg] Merging formats into "./Heavy Traffic on the roads of Delhi.mp4"
Deleting original file ./Heavy Traffic on the roads of Delhi.f135.mp4 (pass -k to keep)
Deleting original file ./Heavy Traffic on the roads of Delhi.mp4.f140 (pass -k to keep)


In [ ]:
!pip install imutils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import time
import cv2
import imutils
from imutils.video import FPS
from imutils.video import VideoStream



INPUT_FILE='newanimal.mp4'
OUTPUT_FILE='newanimaloutput.avi'
LABELS_FILE='coco.names'
CONFIG_FILE='yolov4.cfg'
WEIGHTS_FILE='yolov4.weights'
CONFIDENCE_THRESHOLD=0.5

H=None
W=None

fps = FPS().start()

fourcc = cv2.VideoWriter_fourcc(*"MJPG")
writer = cv2.VideoWriter(OUTPUT_FILE, fourcc, 30, (800, 600), True)

LABELS = open(LABELS_FILE).read().strip().split("\n")

np.random.seed(4)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),dtype="uint8")


net = cv2.dnn.readNetFromDarknet(CONFIG_FILE, WEIGHTS_FILE)

vs = cv2.VideoCapture(INPUT_FILE)


# determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]
cnt =0;
while True:
	cnt+=1
	print ("Frame number", cnt)
	try:
		(grabbed, image) = vs.read()
		if not grabbed:
			break
	except:
		break
	blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
	net.setInput(blob)
	if W is None or H is None:
		(H, W) = image.shape[:2]
	layerOutputs = net.forward(ln)



	# initialize our lists of detected bounding boxes, confidences, and
	# class IDs, respectively
	boxes = []
	confidences = []
	classIDs = []

	# loop over each of the layer outputs
	for output in layerOutputs:
		# loop over each of the detections
		for detection in output:
			# extract the class ID and confidence (i.e., probability) of
			# the current object detection
			scores = detection[5:]
			classID = np.argmax(scores)
			confidence = scores[classID]

			# filter out weak predictions by ensuring the detected
			# probability is greater than the minimum probability
			if confidence > CONFIDENCE_THRESHOLD:
				# scale the bounding box coordinates back relative to the
				# size of the image, keeping in mind that YOLO actually
				# returns the center (x, y)-coordinates of the bounding
				# box followed by the boxes' width and height
				box = detection[0:4] * np.array([W, H, W, H])
				(centerX, centerY, width, height) = box.astype("int")

				# use the center (x, y)-coordinates to derive the top and
				# and left corner of the bounding box
				x = int(centerX - (width / 2))
				y = int(centerY - (height / 2))

				# update our list of bounding box coordinates, confidences,
				# and class IDs
				boxes.append([x, y, int(width), int(height)])
				confidences.append(float(confidence))
				classIDs.append(classID)

	# apply non-maxima suppression to suppress weak, overlapping bounding
	# boxes
	idxs = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE_THRESHOLD, CONFIDENCE_THRESHOLD)

	# ensure at least one detection exists
	if len(idxs) > 0:
		# loop over the indexes we are keeping
		for i in idxs.flatten():
			# extract the bounding box coordinates
			(x, y) = (boxes[i][0], boxes[i][1])
			(w, h) = (boxes[i][2], boxes[i][3])

			color = [int(c) for c in COLORS[classIDs[i]]]

			cv2.rectangle(image, (x, y), (x + w, y + h), color, 4)
			text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
			cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_PLAIN, 0.5, color, 2)

	# show the output image
	#cv2_imshow("output") 
	cv2.resize(image, (800,600))
	writer.write(cv2.resize(image, (800,600)))
	fps.update()
	key = cv2.waitKey(1) & 0xFF
	if key == ord("q"):
		break
		#return

fps.stop()

print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

# do a bit of cleanup
cv2.destroyAllWindows()

# release the file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()


Frame number 1
Frame number 2
Frame number 3
Frame number 4
Frame number 5
Frame number 6
Frame number 7
Frame number 8
Frame number 9
Frame number 10
Frame number 11
Frame number 12
Frame number 13
Frame number 14
Frame number 15
Frame number 16
Frame number 17
Frame number 18
Frame number 19
Frame number 20
Frame number 21
Frame number 22
Frame number 23
Frame number 24
Frame number 25
Frame number 26
Frame number 27
Frame number 28
Frame number 29
Frame number 30
Frame number 31
Frame number 32
Frame number 33
Frame number 34
Frame number 35
Frame number 36
Frame number 37
Frame number 38
Frame number 39
Frame number 40
Frame number 41
Frame number 42
Frame number 43
Frame number 44
Frame number 45
Frame number 46
Frame number 47
Frame number 48
Frame number 49
Frame number 50
Frame number 51
Frame number 52
Frame number 53
Frame number 54
Frame number 55
Frame number 56
Frame number 57
Frame number 58
Frame number 59
Frame number 60
Frame number 61
Frame number 62
Frame number 63
F

In [ ]:
'person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow'


In [ ]:
import csv

In [ ]:
%cd '/content/drive/MyDrive/pedestrian'

/content/drive/MyDrive/pedestrian


In [ ]:
f = open('/content/drive/MyDrive/pedestrian/data.csv', 'w')

In [ ]:
writer = csv.writer(f)

In [ ]:
fields = ['classID', 'confidences', 'X', 'Y', 'W', 'H']

In [ ]:
writer.writerow(fields)

29

In [ ]:
def annotate(self):
    if("annotated" in self.video_path):
        messagebox.showinfo("Error", "You can't annotate an annotated video!")
    elif(self.mode == "V" and not self.video_path is None):
        fourcc = cv2.VideoWriter_fourcc(*'MP4V')
        time = datetime.datetime.now().strftime('%Y-%m-%d %H_%M_%S')
        path = 'output/videos/annotated_' + time + '_output.mp4'
        out = cv2.VideoWriter(path, fourcc, 20.0, (960, 540))
        self.rewind()
        NUM_CLASSES = 4
        detection_graph = tf.Graph()
        with detection_graph.as_default():
            od_graph_def = tf.compat.v1.GraphDef()
            with tf.io.gfile.GFile(self.model_graph, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')
        lmap = label_map_util.load_labelmap(self.label_map)
        categories = label_map_util.convert_label_map_to_categories(lmap, max_num_classes=NUM_CLASSES, use_display_name=True)
        category_index = label_map_util.create_category_index(categories)

        with detection_graph.as_default():
            with tf.compat.v1.Session(graph=detection_graph) as sess:
                while not self.currentFrame is None:
                    image_np = self.get_just_frame()
                    if(image_np is None):
                        break
                    image_np_expanded = np.expand_dims(image_np, axis=0)

                    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

                    self.boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

                    self.scores = detection_graph.get_tensor_by_name('detection_scores:0')

                    classes = detection_graph.get_tensor_by_name('detection_classes:0')

                    num_detections = detection_graph.get_tensor_by_name(
                        'num_detections:0')

                    (self.boxes, self.scores, classes, num_detections) = sess.run(
                        [self.boxes, self.scores, classes, num_detections],
                        feed_dict={image_tensor: image_np_expanded})

                    vis_util.visualize_boxes_and_labels_on_image_array(
                        image_np,
                        np.squeeze(self.boxes),
                        np.squeeze(classes).astype(np.int32),
                        np.squeeze(self.scores),
                        category_index,
                        use_normalized_coordinates=True,
                        line_thickness=2)

                    # Display output
                    out.write(image_np)
        self.video.release()
        out.release()
        self.video = None
        self.set_video_path(path)
        self.video = cv2.VideoCapture(self.video_path)
        if(not self.video.isOpened()):
            raise ValueError("Unable to open video source", self.video_path)
        ret, frame = self.get_frame()
        if(ret and not frame is None):
            self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(frame))  
            self.canvas.create_image(0, 0, image = self.photo, anchor = NW)
        MsgBox = tk.messagebox.askquestion ('Export to CSV','Do you want to export the video to CSV?',icon = 'warning')
        if MsgBox == 'yes':
           self.export_CSV(self.boxes, self.scores, classes, category_index)
    if(self.video_path is None):
        messagebox.showinfo("Error", "No video selected")

def export_CSV(self, boxes, scores, classes, category_index):
    if (boxes is None):
        messagebox.showinfo("Error", "No boxes, you must\nannotate the video first")
        return

    time = datetime.datetime.now().strftime('%Y-%m-%d %H_%M_%S')
    path = 'output/csv/' + time + '_output.csv'
    print(boxes)

    with open(path, 'wb') as write_file:
        writer = csv.writer(write_file)

        for box, score, predicted_class in zip(np.squeeze(boxes), np.squeeze(scores), classes):
            if(score > 0.5):
                print("ymin={}, xmin={}, ymax={}, xmax{}".format(box[0]*540,box[1]*960,box[2]*540,box[3]*960))    
                writer.writerow([box[0], box[1], box[2], box[3], category_index[predicted_class]['name']])


In [ ]:
f.close()

In [ ]:
https://www.youtube.com/watch?v=LoAYJNINWmU

In [ ]:
https://www.youtube.com/watch?v=pU8ThDYZcCc

In [ ]:
https://www.youtube.com/watch?v=7HaJArMDKgI